In [3]:
import os
import re
import subprocess

import mgitools.bsub as bsub
import mgitools.os_helpers as os_helpers

In [1]:
BAM_MAP_LOCATION = '/gscuser/estorrs/CPTAC3.catalog/MGI.BamMap.dat'
# BAM_MAP_LOCATION = '/gscuser/estorrs/gbm.bobo.bammap.dat'
EXECUTION_DIR = '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/gbm/test/execution'
LOG_DIR = '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/gbm/test/indexing'






## heirarchy and symlinking

In [4]:
f = open(BAM_MAP_LOCATION)
sample_data = {}


identifiers = [r'WXS', 'hg38', 'BAM', r'blood_normal|tumor', 'GBM']
# identifiers = ['blood_normal']


# filter based on identifiers and get sample name and location
for line in f:
    is_valid = True
    for identifier in identifiers:
        if not re.findall(identifier, line):
            is_valid = False
            break

    if is_valid:
        pieces = line.strip().split('\t')

        sample = pieces[1]
        technology = pieces[3]
        sample_type = pieces[4]
        fp = pieces[5]
        
        if sample_type == 'blood_normal':
            sample = sample + '.N'
        else:
            sample = sample + '.T'
        
        if sample in sample_data:
            sample_data[sample]
        
        if sample in sample_data:
            d = sample_data[sample]
            if d['technology'].lower() != 'wxs' and technology.lower() == 'wxs':
                sample_data[sample] = {'technology': technology, 'fp': fp}
        else:
            sample_data[sample] = {'technology': technology, 'fp': fp}

len(sample_data), list(sample_data.items())[:5]

(120,
 [('C3L-00016.N',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/fb7e7564-2179-4c8c-8a4c-ed50a4205a4b/65158bcf-b42e-4e35-aa37-aad195b41e03_gdc_realn.bam'}),
  ('C3L-00016.T',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/89be0404-faf9-42f7-8e21-9ad3416a1694/0eb51bef-799b-4c17-89dc-b5e15f13d7bc_gdc_realn.bam'}),
  ('C3L-00019.N',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/60d54f61-0d56-45f9-b6a7-93e6c66af8c0/383c9539-9c57-47c4-90f3-c0453068ff6f_gdc_realn.bam'}),
  ('C3L-00019.T',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/a2b737a0-4c3a-4b5c-808b-21078d18fb8c/8cb0498b-f2ca-40d3-9850-0354f0c53f1e_gdc_realn.bam'}),
  ('C3L-00104.N',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/cd8ff448-91ea-44b2-9293-0c89f3b6c7a3/27b12d40-946d-4fc4-b336-b0edd9bcaa3c_wxs_gdc_realn.bam'})])

In [5]:
sample_to_fps = {}
for sample, d in sample_data.items():
    s_id = sample[:-2]
    
    if s_id not in sample_to_fps:
        sample_to_fps[s_id] = {}
    
    if sample[-1] == 'N':
        sample_to_fps[s_id]['normal'] = d['fp']
    else:
        sample_to_fps[s_id]['tumor'] = d['fp']
len(sample_to_fps), list(sample_to_fps.items())[:5]

(60,
 [('C3L-00016',
   {'normal': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/fb7e7564-2179-4c8c-8a4c-ed50a4205a4b/65158bcf-b42e-4e35-aa37-aad195b41e03_gdc_realn.bam',
    'tumor': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/89be0404-faf9-42f7-8e21-9ad3416a1694/0eb51bef-799b-4c17-89dc-b5e15f13d7bc_gdc_realn.bam'}),
  ('C3L-00019',
   {'normal': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/60d54f61-0d56-45f9-b6a7-93e6c66af8c0/383c9539-9c57-47c4-90f3-c0453068ff6f_gdc_realn.bam',
    'tumor': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/a2b737a0-4c3a-4b5c-808b-21078d18fb8c/8cb0498b-f2ca-40d3-9850-0354f0c53f1e_gdc_realn.bam'}),
  ('C3L-00104',
   {'normal': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/cd8ff448-91ea-44b2-9293-0c89f3b6c7a3/27b12d40-946d-4fc4-b336-b0edd9bcaa3c_wxs_gdc_realn.bam',
    'tumor': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/fc6acfc5-97f2-405f-9fb7-7f1b6c1dce00/d36cf415-f716-4e50-a949-a06459cd5277_wxs_gdc_realn.bam'}),
  ('C3L-00265',
  

In [6]:
for sample, d in sample_to_fps.items():
    # make directory for sample
    os.mkdir(os.path.join(EXECUTION_DIR, sample))
    
    # link files in directory
    for k, fp in d.items():
        if k == 'normal':
            tool_args = ('ln', '-s', fp, os.path.join(EXECUTION_DIR, sample, f'{sample}.N.bam'))
        else:
            tool_args = ('ln', '-s', fp, os.path.join(EXECUTION_DIR, sample, f'{sample}.T.bam'))
            
        subprocess.check_output(tool_args)

## indexing

In [18]:
regex = r'.bam$'
    
fps = list(os_helpers.listfiles(EXECUTION_DIR,
                regex=regex))

sample_fps_tups = [(fp.split('/')[-1].replace('.bam', ''), fp) for fp in fps]

len(sample_fps_tups), sample_fps_tups[:5]

(154,
 [('C3N-01997.N',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-01997/C3N-01997.N.bam'),
  ('C3N-01997.T',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-01997/C3N-01997.T.bam'),
  ('C3N-00514.T',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-00514/C3N-00514.T.bam'),
  ('C3N-00514.N',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-00514/C3N-00514.N.bam'),
  ('C3N-01169.N',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-01169/C3N-01169.N.bam')])

In [19]:
image = 'estorrs/slate:0.0.2'

# os.mkdir(LOG_DIR)

commands, log_files = [], []
for s_id, fp in sample_fps_tups:

    command = f'samtools index {fp}'
    log_fp = os.path.join(LOG_DIR, s_id + '.log')

    commands.append(command)
    log_files.append(log_fp)
        
bsub.generate_bsub_bash_script(commands, image, os.path.join(LOG_DIR, 'commands.sh'),
                              min_memory=4, max_memory=5, log_files=log_files)

## writing executables